# Phrase retrieval

### Biword retrieval

### Positional indexes

In [ ]:
# Open file and create vocab
vocab = []
docs = []
filename = "21-most-cited-machine-learning-papers_titles.txt"

with open(filename, encoding='utf-8') as f:
    lines = f.readlines()
    for line in lines:
        # Word tokenization 

        title = line.split("|")[0].lower()
        docs.append(title)
        for word in title.split():
            if word not in vocab:
                vocab.append(word)


print(len(vocab), "\n", vocab,"\n", len(docs), "\n", docs)

In [3]:
# biword retrieval
import re

def preprocess_text(text):

    # Lowercase and remove punctuation
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    return text.split()

def build_biword_index(documents):
    biword_index = {}

    for doc_id, document in enumerate(documents):
        words = preprocess_text(document)
        # Create biwords from consecutive words
        for i in range(len(words) - 1):
            biword = (words[i], words[i + 1])
            if biword in biword_index.keys():
                biword_index[biword] += [doc_id]
            else:
                biword_index[biword] = [doc_id]
    
    return biword_index

def search_biword_index(biword_index, query):
    words = preprocess_text(query)
    if len(words) < 2:
        return set()  # No biwords in query
    query_biwords = [(words[i], words[i + 1]) for i in range(len(words) - 1)]
    
    # Find the intersection of documents containing all query biwords
    result_docs = None
    for biword in query_biwords:
        if biword in biword_index:
            if result_docs is None:
                result_docs = biword_index[biword]
            else:
                result_docs = result_docs.intersection(biword_index[biword])
        else:
            return set()  # No match for a biword

    return result_docs if result_docs else set()


# Build the biword index
biword_index = build_biword_index(docs)

# Perform a search
query = "deep learning"
result = search_biword_index(biword_index, query)

print("Query:", query)
print("Matching Document IDs:", result)

Query: deep learning
Matching Document IDs: [6]


In [4]:
# positional indexes
from collections import defaultdict

def preprocess_text(text):
    import re
    # Lowercase and remove punctuation
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    return text.split()

def build_positional_index(documents):

    positional_index = defaultdict(list)

    for doc_id, document in enumerate(documents):
        words = preprocess_text(document)
        for pos, word in enumerate(words):
            # Append the position to the term's entry
            if positional_index[word] and positional_index[word][-1][0] == doc_id:
                positional_index[word][-1][1].append(pos)
            else:
                positional_index[word].append((doc_id, [pos]))

    return positional_index

def search_phrase(positional_index, query, documents):

    words = preprocess_text(query)
    if not words:
        return set()

    # Find the list of (doc_id, positions) for each word in the query
    word_positions = [positional_index.get(word, []) for word in words]
    if not all(word_positions):
        return set()

    # Find documents containing all words with correct relative positions
    result_docs = set(doc_id for doc_id, _ in word_positions[0])
    for i in range(1, len(words)):
        next_result_docs = set()
        for doc_id, positions in word_positions[i]:
            for prev_doc_id, prev_positions in word_positions[i - 1]:
                if doc_id == prev_doc_id:
                    if any(pos + 1 in positions for pos in prev_positions):
                        next_result_docs.add(doc_id)
                        break
        result_docs &= next_result_docs
        if not result_docs:
            return set()

    return result_docs


# Build the biword index
biword_index = build_positional_index(docs)

# Perform a search
query = "deep learning"
result = search_phrase(biword_index, query, docs)

print("Query:", query)
print("Matching Document IDs:", result)

Query: deep learning
Matching Document IDs: {6}


### Homework

In [ ]:
# Homework 1:
# Implement the skip pointer

# Homework 2:
# what is the order of skip pointers in both the best and worst cases.

# What are the advantages and disadvantages of positional and biword retrieval. Compare them in a table.